# Generate multimodal travel time matrix (ttm)

This script is to generate a multimodal ttm for accessibility calculation in _r5_.<br/>
Modes: walking, biking, transit, car

It takes inspiration from https://github.com/rafapereirabr/otp-travel-time-matrix/blob/master/python_script.py

## 1. Variables definition and data import

In [2]:
# definitions
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import tracc
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode
from datetime import timedelta
import matplotlib.pyplot as plt
sys.argv.append(["--max-memory", "8G"])


data_folder = "/Users/azanchetta/OneDrive - The Alan Turing Institute/demoland_data"


# regional level files: (require previous editing)
oas_centroids_file = f"{data_folder}/processed/OA_centroids_TyneWear.gpkg" # used for population origin
oas_file = f"{data_folder}/processed/authorities/OA_TyneWear.gpkg" # needed for visualisation purposes
region_lads_file = f"{data_folder}/processed/authorities/LADs_tynewear.shp" # needed in order to filter greenspace data within the regional boundaries
workingplacezones_centroids_file = f"{data_folder}/processed/authorities/WPZ_centroids_tynewear.gpkg"

# national level files
greenspace_file = f"{data_folder}/raw/accessibility/OS Open Greenspace (GPKG) GB/data/opgrsp_gb.gpkg"
osm_data_file = f"{data_folder}/raw/accessibility/tyne-and-wear-latest.osm.pbf"
gtfs_data_file = f"{data_folder}/raw/accessibility/itm_north_east_gtfs.zip"

In [4]:
# import

# origins (IE output areas, OAs)
oas_centroids = gpd.read_file(oas_centroids_file,
                              layer="OA_centroids_TyneWear")
oas_centroids['id'] = oas_centroids['OA11CD'] # Origin dataset must contain an 'id' column
oas_centroids.head()

# destination data
# green space sites' entrances
gs_entrances = gpd.read_file(greenspace_file,
                        layer = "AccessPoint")
gs_entrances.head()
# WPZ centroids
wpz_centroids = gpd.read_file(workingplacezones_centroids_file,
                              layer = "WPZ_centroids_tynewear")
wpz_centroids.head()
wpz_centroids['id'] = wpz_centroids['wz11cd'] # Destination dataset must contain an 'id' column

# network data
# uploaded in the sequent operation

## 2. ttm generation and playing around

In [5]:
### load in transport network

transport_network = TransportNetwork(
    osm_data_file,
    [
        gtfs_data_file
    ]
)

In [ ]:
# example of ttm generation
ttm_walking_OAtoGS = TravelTimeMatrixComputer(
    transport_network,
    origins=origin_centroids_wgs84,
    destinations=accesspoints_wgs84,
    max_time=dt.timedelta(seconds=900), # restricting travel to 15min
    speed_walking=4.8,
    transport_modes=[LegMode.WALK]
)
ttm_walking_OAtoGS = ttm_walking_OAtoGS.compute_travel_times()
ttm_walking_OAtoGS.head()


In [ ]:
## --------------------------------------------------------------##
## ORIGINAL CODE for OTP ----- TO BE RE-WRITTEN   ---- !!!!!!!!!!!


# Start timing the code
import time
start_time = time.time()


# Create a default request for a given departure time
req = otp.createRequest()
req.setDateTime(2015, 9, 15, 10, 00, 00)  # set departure time
req.setMaxTimeSec(7200)                   # set a limit to maximum travel time (seconds)
req.setModes('WALK,BUS,RAIL')             # define transport mode
req.setClampInitialWait(0)                # clamp the initial wait time to zero
# req.maxWalkDistance = 3000                 # set the maximum distance (in meters) the user is willing to walk
# req.walkSpeed = walkSpeed                 # set average walking speed ( meters ?)
# req.bikeSpeed = bikeSpeed                 # set average cycling speed (miles per hour ?)
# ?ERROR req.setSearchRadiusM(500)                 # set max snapping distance to connect trip origin to street network

# Read Points of Destination - The file points.csv contains the columns GEOID, X and Y.
points = otp.loadCSVPopulation('points.csv', 'Y', 'X')
dests = otp.loadCSVPopulation('points.csv', 'Y', 'X')


# Create a CSV output
matrixCsv = otp.createCSVOutput()
matrixCsv.setHeader([ 'origin', 'destination', 'walk_distance', 'travel_time', 'boardings' ])

# Start Loop
for origin in points:
  print "Processing origin: ", origin
  req.setOrigin(origin)
  spt = router.plan(req)
  if spt is None: continue

  # Evaluate the SPT for all points
  result = spt.eval(dests)
  
  # Add a new row of result in the CSV output
  for r in result:
    matrixCsv.addRow([ origin.getStringData('GEOID'), r.getIndividual().getStringData('GEOID'), r.getWalkDistance() , r.getTime(),  r.getBoardings() ])

# Save the result
matrixCsv.save('traveltime_matrix.csv')

# Stop timing the code
print("Elapsed time was %g seconds" % (time.time() - start_time))